In [1]:
import spikeinterface as si
# import spikeinterface.extractors as se 
# import spikeinterface.preprocessing as spre
# import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
# import spikeinterface.qualitymetrics as sqm
# import spikeinterface.comparison as sc
# import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw

In [2]:
# print(f"SpikeInterface version: {si.__version__}")

In [5]:
# import matplotlib.pyplot as plt
# import numpy as np
from pathlib import Path

# import warnings
# warnings.simplefilter("ignore")

# %matplotlib widget

In [6]:
base_folder = Path(r"/scratch2/weka/wanglab/prevosto/data/sc012/sc012_0123/sc012_0123_001")
file_path = base_folder.joinpath("Record Node 101")

### Extract waveforms

In [ ]:
recording_saved = si.load_extractor(base_folder / "preprocessed")
sorting = sorting_KS25
print(sorting)

In [ ]:
si.extract_waveforms?

In [ ]:
we = si.extract_waveforms(recording_saved, sorting, folder=base_folder / "waveforms", 
                          load_if_exists=False, overwrite=True, **job_kwargs)
print(we)

In [ ]:
waveforms0 = we.get_waveforms(unit_id=0)
print(f"Waveforms shape: {waveforms0.shape}")
template0 = we.get_template(unit_id=0)
print(f"Template shape: {template0.shape}")
all_templates = we.get_all_templates()
print(f"All templates shape: {all_templates.shape}")

In [ ]:
w = sw.plot_unit_templates(we, radius_um=30, backend="ipywidgets")

In [ ]:
for unit in sorting.get_unit_ids():
    waveforms = we.get_waveforms(unit_id=unit)
    spiketrain = sorting.get_unit_spike_train(unit)
    print(f"Unit {unit} - num waveforms: {waveforms.shape[0]} - num spikes: {len(spiketrain)}")

In [ ]:
we_all = si.extract_waveforms(recording_saved, sorting, folder=base_folder / "waveforms_all", 
                              max_spikes_per_unit=None,
                              overwrite=True,
                              **job_kwargs)

In [ ]:
for unit in sorting.get_unit_ids():
    waveforms = we_all.get_waveforms(unit_id=unit)
    spiketrain = sorting.get_unit_spike_train(unit)
    print(f"Unit {unit} - num waveforms: {waveforms.shape[0]} - num spikes: {len(spiketrain)}")

### Postprocessing

#### Sparsity

In [ ]:
spost.get_template_channel_sparsity?

In [ ]:
# example: radius
sparsity_radius = spost.get_template_channel_sparsity(we, method="radius", radius_um=50)
print(sparsity_radius[sorting.unit_ids[0]])

In [ ]:
# example: best
sparsity_best = spost.get_template_channel_sparsity(we, method="best_channels", num_channels=4)
print(sparsity_best[sorting.unit_ids[0]])

In [ ]:
sw.plot_unit_templates(we, sparsity=sparsity_radius, backend="ipywidgets")

In [ ]:
sw.plot_unit_templates(we, sparsity=sparsity_best, backend="ipywidgets")

#### PCA scores

In [ ]:
spost.compute_principal_components?

In [ ]:
pc = spost.compute_principal_components(we, n_components=3,
                                        sparsity=sparsity_radius, 
                                        load_if_exists=False,
                                        n_jobs=job_kwargs["n_jobs"], 
                                        progress_bar=job_kwargs["progress_bar"])

In [ ]:
pc0 = pc.get_projections(unit_id=0)
print(f"PC scores shape: {pc0.shape}")
all_labels, all_pcs = pc.get_all_projections()
print(f"All PC scores shape: {all_pcs.shape}")

#### WaveformExtensions

In [ ]:
we.get_available_extension_names()

In [ ]:
pc = we.load_extension("principal_components")
print(pc)

In [ ]:
all_labels, all_pcs = pc.get_data()
print(all_pcs.shape)

#### Spike amplitudes

In [ ]:
amplitudes = spost.compute_spike_amplitudes(we, outputs="by_unit", load_if_exists=True, 
                                            **job_kwargs)

In [ ]:
amplitudes[0]

In [ ]:
sw.plot_amplitudes(we, backend="ipywidgets")

#### Compute unit and spike locations

In [ ]:
unit_locations = spost.compute_unit_locations(we, method="monopolar_triangulation", load_if_exists=True)
spike_locations = spost.compute_spike_locations(we, method="center_of_mass", load_if_exists=True,
                                                **job_kwargs)

In [ ]:
sw.plot_unit_locations(we, backend="ipywidgets")

In [ ]:
sw.plot_spike_locations(we, max_spikes_per_unit=300, backend="ipywidgets")

#### Compute correlograms

In [ ]:
ccgs, bins = spost.compute_correlograms(we)

In [ ]:
sw.plot_autocorrelograms(we, unit_ids=sorting.unit_ids[:3])
sw.plot_crosscorrelograms(we, unit_ids=sorting.unit_ids[:3])

#### Compute template similarity

In [ ]:
similarity = spost.compute_template_similarity(we)

#### Compute template metrics
For more information about these template metrics, we refer to this [documentation](https://github.com/AllenInstitute/ecephys_spike_sorting/tree/master/ecephys_spike_sorting/modules/mean_waveforms) from the Allen Institute.

In [ ]:
print(spost.get_template_metric_names())

In [ ]:
template_metrics = spost.calculate_template_metrics(we)
display(template_metrics)

In [ ]:
sw.plot_template_metrics(we, include_metrics=["peak_to_valley", "half_width"], 
                         backend="ipywidgets")

In [ ]:
!ls waveforms

#### Quality metrics and curation
For more information about these waveform features, we refer to the [SpikeInterface documentation](https://spikeinterface.readthedocs.io/en/latest/module_qualitymetrics.html) and to this excellent [documentation](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html) from the Allen Institute.

In [ ]:
print(sqm.get_quality_metric_list())
print(sqm.get_quality_pca_metric_list())

In [ ]:
qm = sqm.compute_quality_metrics(we, sparsity=sparsity_radius, verbose=True, 
                                 n_jobs=job_kwargs["n_jobs"])

In [ ]:
display(qm)

In [ ]:
sw.plot_quality_metrics(we, include_metrics=["amplitude_cutoff", "presence_ratio", "isi_violations_ratio", "snr"], 
                        backend="ipywidgets")

#### Automatic curation based on quality metrics

In [ ]:
isi_viol_thresh = 0.2
amp_cutoff_thresh = 0.1

In [ ]:
our_query = f"amplitude_cutoff < {amp_cutoff_thresh} & isi_violations_rate < {isi_viol_thresh}"
print(our_query)

In [ ]:
keep_units = qm.query(our_query)
keep_unit_ids = keep_units.index.values

In [ ]:
sorting_auto_KS25 = sorting.select_units(keep_unit_ids)
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_auto_KS25.get_unit_ids())}")

In [ ]:
we.select_units?

In [ ]:
we_curated = we.select_units(keep_unit_ids, new_folder="waveforms_cur")
# we_curated = we.select_units(keep_unit_ids, new_folder=None, use_relative_path="waveforms_curated")

In [ ]:
print(we_curated)

In [ ]:
we_curated.get_available_extension_names()

# 8. Viewers <a class="anchor" id="viewers"></a>


### Spike sorting comparison

#### Compare two sorters

In [ ]:
comp_KS2_KS25 = sc.compare_two_sorters(sorting_KS2, sorting_KS25, 'KS2', 'KS25')

In [ ]:
sw.plot_agreement_matrix(comp_KS2_KS25)

In [ ]:
comp_KS2_KS25auto = sc.compare_two_sorters(sorting_KS2, sorting_auto_KS25, 'KS2', 'KS25_auto')

In [ ]:
sw.plot_agreement_matrix(comp_KS2_KS25auto)

#### Compare multiple sorters

In [ ]:
mcmp = sc.compare_multiple_sorters([sorting_KS2, sorting_KS25], ['KS2', 'KS25'], 
                                   spiketrain_mode='union', verbose=True)

In [ ]:
w = sw.plot_multicomp_agreement(mcmp)
w = sw.plot_multicomp_agreement_by_sorter(mcmp)

In [ ]:
sw.plot_multicomp_graph(mcmp, draw_labels=False)

In [ ]:
agreement_sorting = mcmp.get_agreement_sorting(minimum_agreement_count=2)
print(agreement_sorting)

In [ ]:
sw.plot_rasters(agreement_sorting)

In [ ]:
# compare consensus and auto
comp_agr_auto = sc.compare_two_sorters(agreement_sorting, sorting_auto_KS25, 'AGR', 'AUTO')

In [ ]:
sw.plot_agreement_matrix(comp_agr_auto)